### Download Dataset

In [1]:
import soundata

dataset = soundata.initialize('urbansound8k', data_home="./dataset")
dataset.download()  # download the dataset
dataset.validate()  # validate that all the expected files are there

example_clip = dataset.choice_clip()  # choose a random example clip
print(example_clip)  # see the available data


INFO: Downloading ['all', 'index']. Index is being stored in C:\Users\filip\miniconda3\envs\acii\Lib\site-packages\soundata\datasets\indexes, and the rest of files in ./dataset
INFO: [all] downloading UrbanSound8K.tar.gz
5.61GB [1:10:24, 1.43MB/s]                             
INFO: [index] downloading urbansound8k_index_1.0.json
1.15MB [00:02, 564kB/s]                             
100%|██████████| 8732/8732 [02:03<00:00, 70.61it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


Clip(
  audio_path="./dataset\audio/fold1/162540-1-1-0.wav",
  clip_id="162540-1-1-0",
  audio: The clip's audio
            * np.ndarray - audio signal
            * float - sample rate,
  class_id: The clip's class id.
            * int - integer representation of the class label (0-9). See Dataset Info in the documentation for mapping,
  class_label: The clip's class label.
            * str - string class name: air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, street_music,
  fold: The clip's fold.
            * int - fold number (1-10) to which this clip is allocated. Use these folds for cross validation,
  freesound_end_time: The clip's end time in Freesound.
            * float - end time in seconds of the clip in the original freesound recording,
  freesound_id: The clip's Freesound ID.
            * str - ID of the freesound.org recording from which this clip was taken,
  freesound_start_time: The clip's start time in 

Enquanto só estamos a testar e não a rodar para a entrega do trabalho testar em apenas 1 fold e não nos 10 uma vez que pode demorar demasiado tempo

**Bonus** - Verificar a robustes da rede, ou seja, verificar que alterações se pode fazer ao input até que a rede começe a falhar